In [1]:
from mpi4py import MPI
from manapy.ast import Variable
from manapy.base.base import Struct
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.solvers.advec.tools_utils import initialisation_gaussian_2d
import numpy as np
import time
from numba import cuda

import matplotlib.pyplot as plt

from timeit import default_timer as timer
from manapy.cuda.utils import VarClass
###############
# test_time
###############

def test_time(iter, fun):
  #fun()
  start_time = timer()
  for _ in range(iter):
    fun()
  end_time = timer()
  elapsed_time = (end_time - start_time) / iter
  print(f"{elapsed_time * 1000:.5f} ms")
  #print(f"{elapsed_time * 1000000:.5f} micros")

###############
# init
###############
def init(dim, mesh_path):
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision="single")
  MeshPartition(mesh_path, dim=dim, conf=running_conf, periodic=[0,0,0])

  running_conf = Struct(backend="numba", signature=True, cache =True, float_precision="single")
  domain = Domain(dim=dim, conf=running_conf)
  ne = Variable(domain=domain)
  u  = Variable(domain=domain)
  v  = Variable(domain=domain)
  w  = Variable(domain=domain)
  
  P = Variable(domain=domain)
  Pinit = 2.0
  cells = domain.cells
  initialisation_gaussian_2d(ne.cell, u.cell, v.cell, P.cell, cells.center, Pinit)

  u.face[:] = 2.
  v.face[:] = 0.
  w.face[:] = 0.
  
  u.interpolate_facetocell()
  v.interpolate_facetocell()
  w.interpolate_facetocell()
  return (domain, ne, u, v, w, P)

In [2]:
dim = 2
mesh_file = "/home/aben-ham/Desktop/work/stage/my_manapy/manapy/mesh/2D/carre.msh"
#mesh_file = "/home/ayoub.hamou/mesh/square.msh"
#mesh_file = "/home/aben-ham/Desktop/work/stage/my_manapy/gpu_accelerator/functions/square.msh"
domain, ne, u, v, w, P = init(dim=dim, mesh_path=mesh_file)

Reading gmsh file ...
Saving partition files ...
Number of Cells: 105826
Number of Vertices: 53314
Local domain contruction ...


In [3]:
#PETScKrylovSolver
from manapy.solvers.ls import PETScKrylovSolver

# put this code to PETScKrylovSolver init function so bypass the import of petsc4py
"""
    # try_imports(['import petsc4py',],
    #             'cannot import petsc4py solver!')
    
    # from petsc4py import PETSc as petsc
    
    #self.petsc = petsc
    self.ksp   = None
    
    self.converged_reasons = {}                                                                                                                                                                             
    # for key, val in six.iteritems(petsc.KSP.ConvergedReason.__dict__):                                                                                                                                 
    #     if isinstance(val, int):                                                                                                                                                                       
    #         self.converged_reasons[val] = key    
"""

conf = Struct(reuse_mtx=True, scheme='diamond', verbose=False, precision="single")
L = PETScKrylovSolver(domain=domain, var=P, conf=conf)

SetUp the Linear system ...


In [4]:
# args list
#? The order is imported of the argument list

VarClass.convert_to_var_class([
    domain.nodes,
    domain.faces,
    domain.cells,
    domain.halos,
    ne,
    domain,
    ne.domain._cells,
    L,
    L.var
])

args = [
  L.domain.faces.cellid,
  L.domain.faces.nodeid,
  L.domain.nodes.vertex,
  L.domain.faces.halofid,
  L.domain.halos.halosext,
  L.domain.nodes.oldname,
  L.domain.cells.volume,
  L.domain.nodes.cellid,
  L.domain.cells.center,
  L.domain.halos.centvol,
  L.domain.nodes.halonid,
  L.domain.nodes.periodicid,
  L.domain.nodes.ghostcenter,
  L.domain.nodes.haloghostcenter,
  L.domain.faces.airDiamond,
  L.domain.nodes.lambda_x,
  L.domain.nodes.lambda_y,
  L.domain.nodes.lambda_z,
  L.domain.nodes.number,
  L.domain.nodes.R_x,
  L.domain.nodes.R_y,
  L.domain.nodes.R_z,
  L.domain.faces.param1,
  L.domain.faces.param2,
  L.domain.faces.param3,
  L.domain.faces.param4,
  L.domain.cells.shift,
  L.localsize,
  L.domain.cells.loctoglob,
  L.var.BCdirichlet,
  L._data,
  L._row,
  L._col,
  L.matrixinnerfaces,
  L.domain.halofaces,
  L.var.dirichletfaces,
]



---------------------------------
---------------------------------
can't get attr _nbnodes => _nbnodes
can't get attr nbnodes => _nbnodes
can't set attr for R_x => can't set attribute
can't set attr for R_y => can't set attribute
can't set attr for R_z => can't set attribute
set att: _R_x
set att: _R_y
set att: _R_z
set att: _cellid
set att: _ghostcenter
set att: _ghostfaceinfo
set att: _ghostid
set att: _haloghostcenter
set att: _haloghostfaceinfo
set att: _haloghostid
set att: _halonid
set att: _lambda_x
set att: _lambda_y
set att: _lambda_z
set att: _loctoglob
set att: _name
set att: _number
set att: _oldname
set att: _periodicid
set att: _vertex
can't set attr for cellid => can't set attribute
can't set attr for ghostcenter => can't set attribute
can't set attr for ghostfaceinfo => can't set attribute
can't set attr for ghostid => can't set attribute
can't set attr for haloghostcenter => can't set attribute
can't set attr for haloghostfaceinfo => can't set attribute
can't set attr

In [5]:
from manapy.ast.functions2d import get_triplet_2d as cpu_function
from manapy.cuda.manapy.ast.cuda_functions2d import get_kernel_get_triplet_2d as gpu_function

numba_fun = domain.backend.compile(cpu_function, echo=True, signature=True)
cuda_fun = gpu_function()

compile get_triplet_2d to cpu => signature=(int32[:,:], int32[:,:], float32[:,:], int32[:], int32[:,:], int32[:], float32[:], int32[:,:], float32[:,:], float32[:,:], int32[:,:], int32[:,:], float32[:,:,:], float32[:,:,:], float32[:], float32[:], float32[:], float32[:], int32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:,:], int32, int32[:], int32[:], float32[:], int32[:], int32[:], int32[:], int32[:], int32[:])
compile device_search_element to cuda => signature=int32(int32[:], int32)
compile kernel_get_triplet_2d to cuda => signature=void(int32[:,:], int32[:,:], float32[:,:], int32[:], int32[:,:], int32[:], float32[:], int32[:,:], float32[:,:], float32[:,:], int32[:,:], int32[:,:], float32[:,:,:], float32[:,:,:], float32[:], float32[:], float32[:], float32[:], int32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:,:], int32, int32[:], int32[:], float32[:], int32[:], int32[:], int32[:], 

In [6]:
numba_fun(*args)
%timeit numba_fun(*args)

In [7]:
cuda_fun(*args)
%timeit cuda_fun(*args)

CUDADispatcher(<function get_kernel_get_triplet_2d.<locals>.kernel_get_triplet_2d at 0x7dcbb660e820>) is called
VarClass<int32, (159139, 2)>
VarClass<int32, (159139, 3)>
VarClass<float32, (53314, 4)>
VarClass<int32, (159139,)>
VarClass<int32, (2, 2)>
VarClass<int32, (53314,)>
VarClass<float32, (105826,)>
VarClass<int32, (53314, 10)>
VarClass<float32, (105826, 3)>
VarClass<float32, (2, 2)>
VarClass<int32, (53314, 2)>
VarClass<int32, (53314, 2)>
VarClass<float32, (53314, 2, 5)>
VarClass<float32, (53314, 1, 5)>
VarClass<float32, (159139,)>
VarClass<float32, (53314,)>
VarClass<float32, (53314,)>
VarClass<float32, (53314,)>
VarClass<int32, (53314,)>
VarClass<float32, (53314,)>
VarClass<float32, (53314,)>
VarClass<float32, (53314,)>
VarClass<float32, (159139,)>
VarClass<float32, (159139,)>
VarClass<float32, (159139,)>
VarClass<float32, (159139,)>
VarClass<float32, (105826, 3)>
<class 'numpy.int32'>
VarClass<int32, (105826,)>
VarClass<int32, (0,)>
VarClass<float32, (4496676,)>
VarClass<int32,

/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [11]:
# verify

gpu_res = VarClass.to_device(args[30]).copy_to_host() + VarClass.to_device(args[31]).copy_to_host() + VarClass.to_device(args[32]).copy_to_host()
cpu_res = args[30] + args[31] + args[32]
print(np.sum(cpu_res), " ==? ", np.sum(gpu_res))
gpu_res.sort()
cpu_res.sort()

np.testing.assert_almost_equal(gpu_res, cpu_res, decimal=1)


469076632606.00415  ==?  469076632606.08545


In [ ]:
def f2():
  pass

def f3():
  pass

def f1():
  f2(a,b)
  f3()
